## Predizendo Note de games na loja da Steam

#### Nome: Luciano A S Contri
#### Matrícula: 1500596

    Banco de dados escolhido: Video Games on Steam [in JSON]
    Url: https://www.kaggle.com/datasets/sujaykapadnis/games-on-steam?select=steamdb.min.json  

#### Importando Bibliotecas a serem utilizadas

In [29]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.linear_model import LinearRegression
from sklearn.metrics import  mean_squared_error, mean_absolute_error
import numpy as np


## Análise descritiva e preparação dos dados:

In [30]:
df = pd.read_json('steamdb.json')
df.head(10)

#### Primeiramente precisamos testar a normaliodade da nossa variável dependente

In [31]:
plt.figure(figsize=(20, 10))
df['store_uscore'].dropna().value_counts().sort_index().plot(kind='bar')
plt.xlabel('Score')
plt.ylabel('Contagem')
plt.title('Distribuição de Score')
plt.show()

# Gráfico Q-Q
plt.figure(figsize=(10, 6))
stats.probplot(df['store_uscore'].dropna(), dist="norm", plot=plt)
plt.title('Gráfico Q-Q para store_uscore')
plt.show()

# Teste de Shapiro-Wilk
stat, p = stats.shapiro(df['store_uscore'].dropna())
print('Estatísticas do teste Shapiro-Wilk:', stat)
print('P-valor:', p)

A distribuição parece ser relativamente normal e passar no teste de shapiro, mas precisamos dar cabo de outliers.

#### Corrigindo outliers:

In [32]:
limite_frequencia = df['store_uscore'].value_counts().quantile(0.95)

valores_para_manter = df['store_uscore'].value_counts()[df['store_uscore'].value_counts() < limite_frequencia].index
df = df[df['store_uscore'].isin(valores_para_manter)]


plt.figure(figsize=(20, 10))
df['store_uscore'].dropna().value_counts().sort_index().plot(kind='bar')
plt.xlabel('Score')
plt.ylabel('Contagem')
plt.title('Distribuição de Score')
plt.show()


Agora apenas removendo diretamente para melhorar como o 33.

In [33]:
df = df[df['store_uscore'] != 33]


In [34]:
plt.figure(figsize=(20, 10))
df['store_uscore'].dropna().value_counts().sort_index().plot(kind='bar')
plt.xlabel('Score')
plt.ylabel('Contagem')
plt.title('Distribuição de Score')
plt.show()

# Teste de Shapiro-Wilk
stat, p = stats.shapiro(df['store_uscore'].dropna())
print('Estatísticas do teste Shapiro-Wilk:', stat)
print('P-valor:', p)

##### Pronto agora sim temos uma variável normalizada, apesar

#### Tratando os tipos e colunas úteis.

In [35]:
df['published_store'] = pd.to_datetime(df['published_store'])
df.info()

    A base de dados têm bastante indices nulos e colunas que não nos dão muita informação como URL do game.
    Precisamos retirar itens chave e manter itens que possam ter alguma relevância mas não tenham tantos NaN.

In [36]:
# eliminando colunas que claramente têm muitos valores nulos e pouca correlação com a variável dependente.
indices_to_drop = [0,1,2,5,6,7,8,9,10,11,14,16,17,18,19,24,26,28,30,33,36,39,40,41,42]

# Obter os nomes das colunas correspondentes aos índices
col_names_to_drop = df.columns[indices_to_drop]

# Remover as colunas
df = df.drop(columns=col_names_to_drop)
df['Idade_do_Produto'] = (datetime.now() - df['published_store']).dt.days
df = df.drop(columns= 'published_store')

In [37]:
df.info()

In [38]:
df.nunique()

    Podemos observar que 'gfq_difficulty' e 'platforms' são categóricas, já as outras são ou float ou multivaloradas.

In [39]:
df['platforms'] = df['platforms'].astype('category')

#### Tratando valores nulos.

In [40]:
df.isna().sum()

    precisamos retirar todos os valores nulos da variável dependente antes de prosseguir com as demais.

In [41]:
df = df.dropna(subset='store_uscore')
print(df.isna().sum())
print(df.shape)
print(df.dropna().shape)

    Podemos observar que retirando todos valores nulos, nosso dataset cai muito, podemos tentar preencher com a média os valores em float.

In [42]:
# Lista de colunas do tipo float64 para substituir NaNs pela média
colunas_float64 = ['full_price', 'current_price', 'achievements', 'gfq_rating',
                   'gfq_length', 'stsp_owners', 'stsp_mdntime', 'hltb_single',
                   'hltb_complete', 'meta_score', 'meta_uscore', 'igdb_score',
                   'igdb_uscore', 'igdb_popularity', 'Idade_do_Produto']

# Substituindo NaNs por médias nas colunas especificadas
for coluna in colunas_float64:
    media = df[coluna].mean()
    df[coluna] = df[coluna].fillna(media)
print(df.isna().sum())
print(df.shape)
print(df.dropna().shape)

Já melhorou bastante mas como categories, genres e tags são multivalorados com interseção entre si podemos ao expandi-los tornar todos tags com colunas unicas booleanas excluindo as 3, portanto precisamos tratar apenas gfq_difficulty.

In [43]:
df['gfq_difficulty'].value_counts()

Podemos considerar que "Simple" é o nível de dificuldade mais baixo, seguido por "Easy", "Just Right" como um nível intermediário, "Tough" como mais difícil e "Unforgiving" como o mais difícil.
    
Com isso podemos categorizar a coluna gfq_difficulty em um float de 1 a 9:
1 - Simple
2 - Simple-Easy
3 - Easy
4 - Easy-Just Right
5 - Just Right
6 - Just Right-Tough
7 - Tough
8 - Tough-Unforgiving
9 - Unforgiving

In [44]:
mapeamento_dificuldade = {
    "Simple": 1,
    "Simple-Easy": 2,
    "Easy": 3,
    "Easy-Just Right": 4,
    "Just Right": 5,
    "Just Right-Tough": 6,
    "Tough": 7,
    "Tough-Unforgiving": 8,
    "Unforgiving": 9
}

# Aplicando o mapeamento à coluna gfq_difficulty
df['gfq_difficulty'] = df['gfq_difficulty'].map(mapeamento_dificuldade)
df['gfq_difficulty'].head()

Com isso agora podemos inferir a média nos índices faltantes.

In [45]:
media = round(df['gfq_difficulty'].mean())
df['gfq_difficulty'] = df['gfq_difficulty'].fillna(media)
print(df.isna().sum())
print(df.shape)
print(df.dropna().shape)

 Tudo certo, agora vamos para as multivaloradas como podemos ver embaixo.

In [46]:
df['genres'].unique()

    As colunas ['tags', 'categories', 'genres'] são multivaloradas, para analisarmos cada item
    precisamos expandir em mais colunas.

In [47]:
df['categories'] = df['categories'].apply(lambda x: x.split(',') if x is not None else [])
df['genres'] = df['genres'].apply(lambda x: x.split(',') if x is not None else [])
df['tags'] = df['tags'].apply(lambda x: x.split(',') if x is not None else [])
df.head()

    Claramente alguns itens de cada tabela se sobrepõe, o quê criaria redundância, então temos que trablhar com conjuntos.

In [48]:
# Instanciar o MultiLabelBinarizer para fazer o one-hot encoding
combined_labels = df.apply(lambda x: set(x['categories'] + x['genres'] + x['tags']), axis=1)

# Instanciar o MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Aplicar o one-hot encoding
combined_encoded = pd.DataFrame(mlb.fit_transform(combined_labels), columns=mlb.classes_, index=df.index)

# Concatenar com o DataFrame original
df_encoded = pd.concat([df, combined_encoded], axis=1)


    Agora com essas colunas criadas foi observado 2 colunas com nomes ligeiramente diferentes mas que também são redundantes além de podermos excluir as originais 'tags', 'categories' e 'genres'.

In [49]:
df_encoded['Multi'] = df_encoded['Multi-player'] | df_encoded['Multiplayer']
df_encoded = df_encoded.drop(columns=['Multiplayer', 'Multi-player','tags', 'categories', 'genres'])
print(df_encoded.info())
print(df_encoded.shape)
print(df_encoded.dropna().shape)
df_encoded = df_encoded.dropna()
df_encoded.head()

Preservados quase todos os indices da tabela agora podemos seguir adiante.

## Análise de correlação

In [50]:
corr_matrix = df_encoded.corr()

# Filtrar colunas com correlação acima de um limiar (0.1 neste caso)
# Verificar primeiro se as colunas estão presentes no DataFrame
cols_to_keep = corr_matrix.columns[corr_matrix.abs()['store_uscore'] > 0.15]

# Criar uma nova matriz de correlação apenas com as colunas filtradas
filtered_corr_matrix = df_encoded[cols_to_keep].corr()

# Usar seaborn para criar um heatmap da matriz de correlação filtrada
plt.figure(figsize=(20, 20))
sns.heatmap(filtered_corr_matrix, annot=True, fmt=".2f", cmap='bwr', center= 0.05)

# Mostrar o gráfico
plt.show()

 Observando a matriz de correlação, está claro que a maioria São fracas e a maior é de 0.41 o que indica que temos que fazer um modelo com várias variáveis independentes para termos sucesso.

#### Regressão linear ou polinomial?

#### Regressão linear simples:
   com a variável de maior correlação

In [51]:
X = df_encoded['igdb_uscore']
y = df_encoded['store_uscore']       # Variável dependente
X_sm = sm.add_constant(X)
model_sm = sm.OLS(y, X_sm).fit()
print(model_sm.summary())

In [52]:
y_pred = model_sm.predict(X_sm)
Residuos = y - y_pred
shapiro_test = stats.shapiro(Residuos)
print("Shapiro-Wilk Modelo RLS:", shapiro_test)
bp_test = sm.stats.diagnostic.het_breuschpagan(Residuos, model_sm.model.exog)
print("Breusch-Pagan Modelo RLS:", bp_test)

Não passou em nenhum teste de resíduos. 
Insatisfatório, pode melhorar.

#### Regressão linear com multiplas variáveis:

In [53]:
r_ajustados_modelo = []
f_statistics_modelo  = []
limite = []
residuos_normais = []
for i in range(15):
    corr_matrix = df_encoded.corr(numeric_only=True)
    multi = 0.05 + (0.01 * i)
    limite.append(multi)
    cols_to_keep = corr_matrix.index[corr_matrix.abs()['store_uscore'] > multi]
    cols_to_keep = cols_to_keep.drop('store_uscore')  # Remover a coluna 'score' da lista
    X = df_encoded[cols_to_keep]  # Variáveis independentes
    y = df_encoded['store_uscore']       # Variável dependente
    X_sm = sm.add_constant(X)
    model_sm = sm.OLS(y, X_sm).fit()

    y_pred = model_sm.predict(X_sm)
    Residuos = y - y_pred
    bp_test = sm.stats.diagnostic.het_breuschpagan(Residuos, model_sm.model.exog)
    if bp_test[2] > 0.05:
        residuos_normais.append(multi)

    adj_r_squared = model_sm.rsquared_adj
    f_statistic = model_sm.fvalue

    r_ajustados_modelo.append(adj_r_squared)
    f_statistics_modelo.append(f_statistic)
print('Modelos que passaram no teste de resíduos de pagan:\n')
print(residuos_normais)

In [54]:
plt.figure(figsize=(14, 6))

# Primeiro subplot para R-quadrado ajustado
plt.subplot(1, 2, 1)  # (linhas, colunas, índice do subplot)
plt.plot(limite,r_ajustados_modelo, label='R-quadrado Ajustado', color='blue', marker='o')
plt.title('R-quadrado Ajustado')
plt.xlabel('Iteração (Limiar de Correlação)')
plt.ylabel('R-quadrado Ajustado')
plt.grid(True)
plt.legend()

# Segundo subplot para F-statistic
plt.subplot(1, 2, 2)  # (linhas, colunas, índice do subplot)
plt.plot(limite,f_statistics_modelo, label='F-statistic', color='red', marker='x')
plt.title('F-statistic')
plt.xlabel('Iteração (Limiar de Correlação)')
plt.ylabel('F-statistic')
plt.grid(True)
plt.legend()

# Ajusta o layout para manter tudo organizado
plt.tight_layout()

# Mostra o gráfico
plt.show()

QQ-PLOT do maior R-ajustado e maior F-statistic

In [55]:
cols_to_keep = corr_matrix.index[corr_matrix.abs()['store_uscore'] > 0.05]
cols_to_keep = cols_to_keep.drop('store_uscore')  # Remover a coluna 'score' da lista
X = df_encoded[cols_to_keep]  # Variáveis independentes
y = df_encoded['store_uscore']       # Variável dependente
X_sm = sm.add_constant(X)
model_sm = sm.OLS(y, X_sm).fit()

y_pred = model_sm.predict(X_sm)
Residuos_r_maior = y - y_pred
bp_test = sm.stats.diagnostic.het_breuschpagan(Residuos_r_maior, model_sm.model.exog)

print(bp_test)
print(model_sm.summary())

cols_to_keep = corr_matrix.index[corr_matrix.abs()['store_uscore'] > 0.18]
cols_to_keep = cols_to_keep.drop('store_uscore')  # Remover a coluna 'score' da lista
X = df_encoded[cols_to_keep]     # Variável dependente
X_sm = sm.add_constant(X)
model_sm = sm.OLS(y, X_sm).fit()

y_pred = model_sm.predict(X_sm)
Residuos_r_menor = y - y_pred
bp_test = sm.stats.diagnostic.het_breuschpagan(Residuos_r_menor, model_sm.model.exog)

print(bp_test)
print(model_sm.summary())

plt.figure(figsize=(12, 6))

# Gráfico Q-Q para o primeiro modelo
plt.subplot(1, 2, 1)  # (linhas, colunas, índice do subplot)
sm.qqplot(Residuos_r_maior, line='s', ax=plt.gca())
plt.title('Gráfico Q-Q do Modelo 1')

# Gráfico Q-Q para o segundo modelo
plt.subplot(1, 2, 2)  # (linhas, colunas, índice do subplot)
sm.qqplot(Residuos_r_menor, line='s', ax=plt.gca())
plt.title('Gráfico Q-Q do Modelo 2')

# Mostrar o gráfico
plt.tight_layout()
plt.show()

Ainda aparenta um R-ajustado baixo. O modelo explica pouco mesmo com residuos melhorando um pouco com f-stat maior.

#### Regressão Polinomial:

In [57]:
cols_to_keep = corr_matrix.index[corr_matrix.abs()['store_uscore'] > 0.1]
cols_to_keep = cols_to_keep.drop('store_uscore')  # Remover a coluna 'score' da lista
X = df_encoded[cols_to_keep]  # Variáveis independentes
y = df_encoded['store_uscore']

r_ajustados_modelo_poli = []
f_statistics_modelo_poli = []
residuos_normais = []
degrees = [1,2,3,4]
for degree in degrees:  # Graus 1 a 5
    poly = PolynomialFeatures(degree)
    X_poly = poly.fit_transform(X)

    X_sm_poly = sm.add_constant(X_poly)
    model_sm_poly = sm.OLS(y, X_sm_poly).fit()

    # Extrair R² ajustado e F-statistic
    adj_r_squared = model_sm_poly.rsquared_adj
    f_statistic = model_sm_poly.fvalue
    r_ajustados_modelo_poli.append(adj_r_squared)
    f_statistics_modelo_poli.append(f_statistic)

    y_pred = model_sm_poly.predict(X_sm_poly)
    Residuos = y - y_pred
    bp_test = sm.stats.diagnostic.het_breuschpagan(Residuos, model_sm.model.exog)
    if bp_test[2] > 0.05:
        residuos_normais.append(degree)
print('Grau polinomial que passaram no teste de pagan:\n')
print(residuos_normais)

plt.figure(figsize=(14, 6))

# Primeiro subplot para R-quadrado ajustado
plt.subplot(1, 2, 1)  # (linhas, colunas, índice do subplot)
plt.plot(r_ajustados_modelo_poli, label='R-quadrado Ajustado', color='blue', marker='o')
plt.title('R-quadrado Ajustado')
plt.xlabel('Grau')
plt.ylabel('R-quadrado Ajustado')
plt.grid(True)
plt.legend()

# Segundo subplot para F-statistic
plt.subplot(1, 2, 2)  # (linhas, colunas, índice do subplot)
plt.plot(f_statistics_modelo_poli, label='F-statistic', color='red', marker='x')
plt.title('F-statistic')
plt.xlabel('Grau')
plt.ylabel('F-statistic')
plt.grid(True)
plt.legend()

# Ajusta o layout para manter tudo organizado
plt.tight_layout()

# Mostra o gráfico
plt.show()


In [58]:
poly = PolynomialFeatures(4)
X_poly = poly.fit_transform(X)

X_sm_poly = sm.add_constant(X_poly)
model_sm_poly = sm.OLS(y, X_sm_poly).fit()

y_pred = model_sm_poly.predict(X_sm_poly)
Residuos = y - y_pred
bp_test = sm.stats.diagnostic.het_breuschpagan(Residuos, model_sm.model.exog)
print(bp_test)
print(model_sm_poly.summary())


sm.qqplot(Residuos, line='s')
plt.title('Gráfico Q-Q dos Resíduos')
plt.show()

##### Como podemos observar a correlação dos dados são fracas, testados a Regressão linear simples, com variáveis multiplas e polinomial, a polinomial se saiu melhor.<br> já a Regressão linear simples sequer passou nos testes de residuo o que indica heterocedasticidade.